In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
from time import sleep
import json

In [32]:
## Pass in a URL and it returns the data for that professor
def get_data(url):
    req = requests.get(url).content
    bs = BeautifulSoup(req)
    name = list(bs.find("div", class_="NameTitle__Name-dowf0z-0 cfjPUG").descendants)
    name = f"{name[1]} {name[4]}"
    department = bs.find("a", class_="TeacherDepartment__StyledDepartmentLink-fl79e8-0 fuypDB").b.contents[0]
    school = bs.find("a", href="/school?sid=1074").contents[0]
    tags = np.unique(bs.find_all("span", class_="Tag-bs9vf4-0 hHOVKF"), return_counts=True)
    tags = {i:j for i,j in zip(tags[0], tags[1])}
    rating = bs.find("div", class_="RatingValue__Numerator-qw8sqy-2 liyUjw").contents[0]
    num_ratings = bs.find('a', href="#ratingsList").contents[0]
    feedback = bs.find_all("div", "FeedbackItem__FeedbackNumber-uof32n-1 kkESWs")
    take_again = feedback[0].contents[0][:-1]
    difficulty = feedback[1].contents[0]
    return [name, department, school, tags, rating, num_ratings, take_again, difficulty]


In [3]:
## Example dataframe -> appends data to csv file
pd.DataFrame(data=get_data("https://www.ratemyprofessors.com/professor?tid=2466852"), index=["name", "department", "school", "tags", "rating", "take_again", "difficulty"]).transpose().to_csv(f"professors.csv", mode='a', header=False, index=False)

In [33]:
l = json.load(open("uci_rmp_raw_text.json"))["data"]["search"]["teachers"]["edges"]
ids = [l[i]["node"]["legacyId"] for i in range(len(l))] # id's of RMP pages for UCI

In [ ]:
## Loop that scraped RMP Profs from UCI
for page in ids:
    try:
        pd.DataFrame(data=get_data(f"https://www.ratemyprofessors.com/professor?tid={page}"), index=["name", "department", "school", "tags", "rating", "num_ratings", "take_again", "difficulty"]).transpose().to_csv(f"professors.csv", mode='a', header=False, index=False)
        print('x')
    except Exception as e:
        print(e)
    finally:
        sleep(0.1)
        page += 12

In [195]:
profs = pd.read_csv("merged.csv")
for i in range(profs.shape[0]):
    profs.at[i, "tags"] = {i:j for i,j in eval(profs.at[i, "tags"]).items() if j > 2}

for i in range(profs.shape[0]):
    for tag in profs.at[i, "tags"].keys():
        if tag not in profs.columns: 
            profs[tag] = [0 for j in range(profs.shape[0])]
    for j,k in profs.at[i, "tags"].items():
        profs.at[i, j] = k
    

In [196]:
includes = []
for i in profs.columns[7:]:
    if np.count_nonzero(profs[i]) > 200:
        includes.append(i)

profs_new = profs[list(profs.columns[:7]) + includes].drop(columns=["tags"])

In [198]:
profs_new.to_csv("ready.csv", index=False)

In [103]:
grades = pd.read_csv("grades.csv")[["instructor", "avg_gpa"]]
query = ~grades["avg_gpa"].isnull()
grades = grades[query]
averages = {
    "instructor": [],
    "avg_gpa": []
}
for i in grades["instructor"].unique():
    averages["avg_gpa"].append(grades[grades.instructor == i].mean().iloc[0])
    averages["instructor"].append(i)


/var/folders/p8/x4dqmdxn5wgfnmwr3f9lr_rm0000gn/T/ipykernel_6372/3704766343.py:9: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  averages["avg_gpa"].append(grades[grades.instructor == i].mean().iloc[0])


In [123]:
profs = pd.read_csv("professors.csv")
for i in range(profs.shape[0]):
    names = profs.at[i,"name"].split(' ')
    if len(names) > 1:
        profs.at[i,"name"] = f"{names[-1]}, {names[0][0]}.".upper() 
    else:
        profs.at[i,"name"] = i.upper()

,name,department,school,tags,rating,take_again,difficulty
0,"CAMPBELL, M.",Mathematics,UC Irvine,"{'Clear grading criteria': 2, 'Get ready to re...",3.1,0,3.3
1,"MOELLER, B.",Humanities,UC Irvine,"{'Amazing lectures ': 1, 'Beware of pop quizze...",4.2,N/,2.6
2,"APKARIAN, V.",Chemistry,UC Irvine,"{""Skip class? You won't pass."": 3, 'Test heavy...",2.9,40,3.9
3,"MILES, M.",Art History,UC Irvine,"{'Amazing lectures': 1, 'Amazing lectures ': 1...",4.3,80,3.1
4,"GLAZER, A.",Economics,UC Irvine,"{'ACCESSIBLE OUTSIDE CLASS': 1, 'Caring': 1, '...",2.3,0,3.4
...,...,...,...,...,...,...,...
3594,"SHORT, G.",English,UC Irvine,"{'Accessible outside class': 2, 'Amazing lectu...",4.3,62,3.5
3595,"KING, S.",Chemistry,UC Irvine,"{'Accessible outside class': 1, 'Amazing lectu...",4.0,76,3.7
3596,"BERGHOF, A.",Literature,UC Irvine,"{'Accessible outside class': 3, 'Amazing lectu...",3.6,73,2.8
3597,"EDWARDS, K.",Chemistry,UC Irvine,"{'Clear grading criteria': 1, 'Graded by few t...",2.1,12,4.1


In [117]:
profs.sort_values("name").drop_duplicates("name")


,name,department,school,tags,rating,take_again,difficulty
3099,"(NICK), A.",Biological Sciences,UC Irvine,"{'Accessible outside class': 6, 'Amazing lectu...",5.0,99,3.1
2785,", D.",History,UC Irvine,"{'Accessible outside class': 1, 'Amazing lectu...",4.5,67,2.6
2915,", E.",Political Science,UC Irvine,"{'Clear grading criteria': 1, 'Get ready to re...",3.3,100,3.1
2890,", J.",Social Science,UC Irvine,"{'Accessible outside class': 10, 'Amazing lect...",3.9,54,4.3
2588,", M.",Spanish,UC Irvine,"{'Accessible outside class': 1, 'BEWARE OF POP...",4.7,100,3.7
...,...,...,...,...,...,...,...
2382,"ZISSOS, A.",Classics,UC Irvine,"{'Amazing lectures': 4, 'Amazing lectures ': 3...",4.1,56,2.5
497,"ZITO, S.",History,UC Irvine,{},3.8,N/,2.3
3569,"ZIV, H.",Computer Science,UC Irvine,"{'Caring': 2, 'Clear grading criteria': 1, 'Ge...",2.3,43,2.8
3156,"ZOELLER, C.",Sociology,UC Irvine,"{'Accessible outside class': 2, 'Amazing lectu...",4.6,94,2.4


In [127]:
import difflib
def closest_match(value, target_list):
    closest_value = difflib.get_close_matches(value, target_list, n=1, cutoff=0.6)
    if closest_value:
        return closest_value[0]
    else:
        return None

def replace_value(value_list, target_list):
    result = []
    for value in value_list:
        closest = closest_match(value, target_list)
        if closest:
            result.append(closest)
        else:
            result.append(value)
    return result

value_list = list(profs["name"])
target_list = list(averages["instructor"])
result = replace_value(value_list, target_list)


{'ABBAS, M.',
 'ABO-SHADI, N.',
 'ADAMS, T.',
 'AGUAS, M.',
 'AL FARUQUE, M.',
 'ALEXANDER, K.',
 'ALLAIRE, M.',
 'ALLEN, C.',
 'AMON, M.',
 'ANDERSEN, B.',
 'ANSELMI II, J.',
 'APELIAN, D.',
 'APKARIAN, A.',
 'ARDONA, H.',
 'ASHOK, PARAMESWARAN.',
 'ATASHI, S.',
 'AWAD, S.',
 'AYALA, F.',
 'AZIZI, E.',
 'BA, B.',
 'BAI, J.',
 'BAKER, V.',
 'BARAM, T.',
 'BARRON, C.',
 'BASOLO, M.',
 'BEIER, K.',
 'BELL, M.',
 'BERNS, M.',
 'BITNER, K.',
 'BLACKSTOCK-BERNSTE.',
 'BOUNDS, D.',
 'BOWLES, C.',
 'BOWMAN, B.',
 'BRAUN, B.',
 'BREITENSTEIN, S.',
 'BRITCH, J.',
 'BURKE, S.',
 'BURRELL, J.',
 'BURT, M.',
 'BURTON, B.',
 'BUSBY, D.',
 'CABRERA, J.',
 'CAMPI, A.',
 'CANAC, N.',
 'CARLO, G.',
 'CARLSON, S.',
 'CARRILLO, C.',
 'CASAVANTES, BRADFOR.',
 'CASTILLO, E.',
 'CASTRO, A.',
 'CEDILLO, R.',
 'CHAHINIAN, T.',
 'CHAN, H.',
 'CHAN, K.',
 'CHAN, S.',
 'CHANDRAMOWLISHWARA.',
 'CHANG, P.',
 'CHAPMAN, D.',
 'CHAVEZ, A.',
 'CHENG, G.',
 'CHENG, Y.',
 'CHERIAN, R.',
 'CHOI, S.',
 'CHOI, Y.',
 'CHRIS

In [129]:
profs

,name,department,school,tags,rating,take_again,difficulty
0,"CAMPBELL, M.",Mathematics,UC Irvine,"{'Clear grading criteria': 2, 'Get ready to re...",3.1,0,3.3
1,"MOELLER, B.",Humanities,UC Irvine,"{'Amazing lectures ': 1, 'Beware of pop quizze...",4.2,N/,2.6
2,"APKARIAN, V.",Chemistry,UC Irvine,"{""Skip class? You won't pass."": 3, 'Test heavy...",2.9,40,3.9
3,"MILES, M.",Art History,UC Irvine,"{'Amazing lectures': 1, 'Amazing lectures ': 1...",4.3,80,3.1
4,"GLAZER, A.",Economics,UC Irvine,"{'ACCESSIBLE OUTSIDE CLASS': 1, 'Caring': 1, '...",2.3,0,3.4
...,...,...,...,...,...,...,...
3594,"SHORT, G.",English,UC Irvine,"{'Accessible outside class': 2, 'Amazing lectu...",4.3,62,3.5
3595,"KING, S.",Chemistry,UC Irvine,"{'Accessible outside class': 1, 'Amazing lectu...",4.0,76,3.7
3596,"BERGHOF, A.",Literature,UC Irvine,"{'Accessible outside class': 3, 'Amazing lectu...",3.6,73,2.8
3597,"EDWARDS, K.",Chemistry,UC Irvine,"{'Clear grading criteria': 1, 'Graded by few t...",2.1,12,4.1


In [10]:
data = pd.read_csv("grades.csv")
data["course_name"] = [f"{i} {j}" for i,j in zip(data["dept_code"], data["number"])]
data = data[["course_name", "avg_gpa"]]

new = {
    "course_name": [],
    "avg_gpa": []
}
courses = pd.unique(data["course_name"])
for i in range(len(courses)):
    new["course_name"].append(courses[i])
    new["avg_gpa"].append(data[data.course_name == courses[i]].mean().iloc[0])

pd.DataFrame(new)[~pd.DataFrame(new)["avg_gpa"].isna()]

/var/folders/p8/x4dqmdxn5wgfnmwr3f9lr_rm0000gn/T/ipykernel_11253/757150161.py:12: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  new["avg_gpa"].append(data[data.course_name == courses[i]].mean().iloc[0])


,course_name,avg_gpa
4,AFAM 40C,3.370000
5,AFAM 138,3.632000
6,ANTHRO 2A,3.168214
7,ANTHRO 2B,2.771481
8,ANTHRO 2D,3.285714
...,...,...
3325,UPPP 127,3.740000
3327,ART 65B,3.720000
3328,GERMAN S1AB,3.730000
3329,GERMAN S1BC,3.640000
